In [93]:
import utils
import ingest
import dotenv
import pandas as pd
from config import load_config as load, load_db_opts, load_polygon_opts
from scrap import _scrap_sp_500_constituents

In [94]:
# Load local dotenv (because not dockerize for now - can be added to image or passed at docker run)
dotenv.load_dotenv(utils.ENV_DOCKER_FILE)
dotenv.load_dotenv(utils.ENV_SECRETS_FILE)
config = load(load_db_opts, load_polygon_opts)

In [95]:
# get tickers
# get each tickers earning dates
# roll 5 days before earning dates values
# get minute graphs for  | 5 days roll | > 10%

In [96]:
# get tickers
from scrap import _scrap_sp_500_constituents

sp500_constituents = _scrap_sp_500_constituents()

In [98]:
# get each tickers earning dates
from scrap import _scrap_previous_earnings
all_earning_dates = pd.DataFrame()

for _, row in sp500_constituents.head(5).iterrows():
    symbol = row["symbol"]
    curr_df = _scrap_previous_earnings(symbol)
    all_earning_dates = pd.concat([all_earning_dates, curr_df])

# filter on past earning dates only
from datetime import datetime

earning_dates = all_earning_dates[all_earning_dates["earnings_date"] < datetime.now()]

In [134]:
# get data for each tickers
from datetime import timedelta
from scrap import _scrap_daily_prices_for_earnings

all_prices_around_earnings = pd.DataFrame()

for _, row in earning_dates.iterrows():
    symbol = row["symbol"]
    earnings_date = row["earnings_date"]
    earnings_date_5_days_before = earnings_date - timedelta(days=6)

    curr_df = _scrap_daily_prices_for_earnings(symbol, earnings_date_5_days_before, earnings_date)
    curr_df["earnings_date"] = row["earnings_date"]
    curr_df["rank_over_group"] = curr_df[["symbol", "earnings_date", "date"]].apply(tuple,axis=1).rank(method="dense", ascending=True).astype(int)

    curr_df = curr_df.pivot(index=["symbol", "earnings_date"], columns=["rank_over_group"], values=["date", "open", "high", "low", "close"])
    
    all_prices_around_earnings = pd.concat([all_prices_around_earnings, curr_df])

print(all_prices_around_earnings.columns)

MultiIndex([( 'date', 1),
            ( 'date', 2),
            ( 'date', 3),
            ( 'date', 4),
            ( 'date', 5),
            ( 'open', 1),
            ( 'open', 2),
            ( 'open', 3),
            ( 'open', 4),
            ( 'open', 5),
            ( 'high', 1),
            ( 'high', 2),
            ( 'high', 3),
            ( 'high', 4),
            ( 'high', 5),
            (  'low', 1),
            (  'low', 2),
            (  'low', 3),
            (  'low', 4),
            (  'low', 5),
            ('close', 1),
            ('close', 2),
            ('close', 3),
            ('close', 4),
            ('close', 5)],
           names=[None, 'rank_over_group'])


In [135]:
all_prices_around_earnings

date                                   \
rank_over_group                     1          2          3          4   
symbol earnings_date                                                     
MMM    2023-07-25 02:00:00 2023-07-18 2023-07-19 2023-07-20 2023-07-21   
       2023-04-25 02:00:00 2023-04-18 2023-04-19 2023-04-20 2023-04-21   
       2023-01-24 01:00:00 2023-01-17 2023-01-18 2023-01-19 2023-01-20   
       2022-10-25 02:00:00 2022-10-18 2022-10-19 2022-10-20 2022-10-21   
       2022-07-26 02:00:00 2022-07-19 2022-07-20 2022-07-21 2022-07-22   
...                               ...        ...        ...        ...   
ABMD   1998-05-28 00:00:00 1998-05-21 1998-05-22 1998-05-26 1998-05-27   
       1998-01-28 00:00:00 1998-01-21 1998-01-22 1998-01-23 1998-01-26   
       1995-05-16 00:00:00 1995-05-09 1995-05-10 1995-05-11 1995-05-12   
       1995-01-24 00:00:00 1995-01-17 1995-01-18 1995-01-19 1995-01-20   
       1994-10-26 00:00:00 1994-10-19 1994-10-20 1994-10-21 1994-10-24   

                                             open                          \
rank_over_group                     5           1           2           3   
symbol earnings_date                                                        
MMM    2023-07-25 02:00:00 2023-07-24  101.620003  102.849998  103.169998   
       2023-04-25 02:00:00 2023-04-24  107.230003  105.699997  105.459999   
       2023-01-24 01:00:00 2023-01-23  129.759995  126.849998       122.0   
       2022-10-25 02:00:00 2022-10-24  115.739998  115.099998  113.980003   
       2022-07-26 02:00:00 2022-07-25  130.399994  132.119995  133.039993   
...                               ...         ...         ...         ...   
ABMD   1998-05-28 00:00:00        NaT        7.94       8.125       8.375   
       1998-01-28 00:00:00 1998-01-27        7.94        7.88        8.13   
       1995-05-16 00:00:00 1995-05-15       4.125       4.125        3.88   
       1995-01-24 00:00:00 1995-01-23        2.63       2.625       2.625   
       1994-10-26 00:00:00 1994-10-25         3.0       3.375       2.875   

                                                    ...         low  \
rank_over_group                      4           5  ...           1   
symbol earnings_date                                ...               
MMM    2023-07-25 02:00:00  104.169998  104.529999  ...  101.349998   
       2023-04-25 02:00:00  104.980003  105.050003  ...  105.870003   
       2023-01-24 01:00:00  118.550003  120.610001  ...  126.389999   
       2022-10-25 02:00:00  113.150002  117.529999  ...  114.449997   
       2022-07-26 02:00:00  134.649994       134.0  ...  130.059998   
...                                ...         ...  ...         ...   
ABMD   1998-05-28 00:00:00        7.38         NaN  ...       7.938   
       1998-01-28 00:00:00        8.19        8.13  ...       7.813   
       1995-05-16 00:00:00        4.25       4.125  ...         4.0   
       1995-01-24 00:00:00        2.75        2.75  ...       2.625   
       1994-10-26 00:00:00       2.875      3.0625  ...       2.875   

                                                                            \
rank_over_group                      2           3           4           5   
symbol earnings_date                                                         
MMM    2023-07-25 02:00:00  102.599998  102.510002  103.220001  103.980003   
       2023-04-25 02:00:00  105.459999  104.110001  104.059998  104.360001   
       2023-01-24 01:00:00  122.260002      118.32  116.760002  120.410004   
       2022-10-25 02:00:00  113.239998  112.889999  113.050003  117.190002   
       2022-07-26 02:00:00  131.850006  132.369995  133.479996  133.779999   
...                                ...         ...         ...         ...   
ABMD   1998-05-28 00:00:00       7.938       7.438       7.375         NaN   
       1998-01-28 00:00:00       7.813       7.875       8.125       8.125   
       1995-05-16 00:00:00        3.75        3.75        3.75  

In [141]:
# compute differences
#earnings_with_lagging_prices["order_per_group"] = 
test = all_prices_around_earnings.reset_index()

In [144]:
test

symbol       earnings_date       date                        \
rank_over_group                                     1          2          3   
0                  MMM 2023-07-25 02:00:00 2023-07-18 2023-07-19 2023-07-20   
1                  MMM 2023-04-25 02:00:00 2023-04-18 2023-04-19 2023-04-20   
2                  MMM 2023-01-24 01:00:00 2023-01-17 2023-01-18 2023-01-19   
3                  MMM 2022-10-25 02:00:00 2022-10-18 2022-10-19 2022-10-20   
4                  MMM 2022-07-26 02:00:00 2022-07-19 2022-07-20 2022-07-21   
..                 ...                 ...        ...        ...        ...   
418               ABMD 1998-05-28 00:00:00 1998-05-21 1998-05-22 1998-05-26   
419               ABMD 1998-01-28 00:00:00 1998-01-21 1998-01-22 1998-01-23   
420               ABMD 1995-05-16 00:00:00 1995-05-09 1995-05-10 1995-05-11   
421               ABMD 1995-01-24 00:00:00 1995-01-17 1995-01-18 1995-01-19   
422               ABMD 1994-10-26 00:00:00 1994-10-19 1994-10-20 1994-10-21   

                                             open                          \
rank_over_group          4          5           1           2           3   
0               2023-07-21 2023-07-24  101.620003  102.849998  103.169998   
1               2023-04-21 2023-04-24  107.230003  105.699997  105.459999   
2               2023-01-20 2023-01-23  129.759995  126.849998       122.0   
3               2022-10-21 2022-10-24  115.739998  115.099998  113.980003   
4               2022-07-22 2022-07-25  130.399994  132.119995  133.039993   
..                     ...        ...         ...         ...         ...   
418             1998-05-27        NaT        7.94       8.125       8.375   
419             1998-01-26 1998-01-27        7.94        7.88        8.13   
420             1995-05-12 1995-05-15       4.125       4.125        3.88   
421             1995-01-20 1995-01-23        2.63       2.625       2.625   
422             1994-10-24 1994-10-25         3.0       3.375       2.875   

                 ...         low                                      \
rank_over_group  ...           1           2           3           4   
0                ...  101.349998  102.599998  102.510002  103.220001   
1                ...  105.870003  105.459999  104.110001  104.059998   
2                ...  126.389999  122.260002      118.32  116.760002   
3                ...  114.449997  113.239998  112.889999  113.050003   
4                ...  130.059998  131.850006  132.369995  133.479996   
..               ...         ...         ...         ...         ...   
418              ...       7.938       7.938       7.438       7.375   
419              ...       7.813       7.813       7.875       8.125   
420              ...         4.0        3.75        3.75        3.75   
421              ...       2.625       2.625       2.625       2.625   
422              ...       2.875       2.875       2.875       2.875   

                                  close                                      \
rank_over_group           5           1           2           3           4   
0                103.980003  103.010002  103.480003  104.290001  104.330002   
1                104.360001  106.269997  106.080002  104.730003  104.480003   
2                120.410004  126.599998      122.75      118.43  120.650002   
3                117.190002  115.949997  114.199997  113.110001  116.809998   
4                133.779999  132.130005  133.009995  133.949997  134.119995   
..                      ...         ...         ...         ...         ...   
418                     NaN       8.125       7.938       7.438       7.438   
419                   8.125         8.0       7.875       8.188       8.219   
420                   4.125         4.0       4.125        4.25       4.125   
421                    2.75       2.782       2.625       2.625       2.625   
422                   2.875         3.0       3.375       2.875       2.875   

                     